### Обязательные библиотеки:
- [Gensim](https://radimrehurek.com/gensim/) — инструмент для решения различных задач NLP (тематическое моделирование, представление текстов, ...);
- [Numpy](http://www.numpy.org) — библиотека для векторных вычислений;
Для выполнения дополнительных заданий (по желанию)
- [Pandas](https://pandas.pydata.org) - библиотека для анализа табличных данных;
- [scikit-learn](http://scikit-learn.org/stable/index.html) — библилиотека с алгоритмами классического машинного обучения;
- [matplotlib](https://matplotlib.org) - библиотека для построения графиков;

-------

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

# Константы
DATA_DIR = 'data'
RANDOM_STATE = 42

### Модель, генерирующая вектора слов

In [3]:
from utils import download_model

download_model(data_dir=DATA_DIR)

Распаковываем архив...
Архив успешно распакован


In [4]:
import gensim

wv_embeddings = gensim.models.fasttext.FastTextKeyedVectors.load(
    DATA_DIR + "/model/model.model"
)

In [16]:
def get_a(model, target_word, topn, result_size):
    candidates = model.most_similar(target_word, topn=topn)
    result = []
    used_prefixes = {target_word.lower()}  
    for w, score in candidates:
        w_lower = w.lower()
        if not w_lower.isalpha() and "-" not in w_lower:
            continue
        if target_word.lower() in w_lower:
            continue
        if w_lower[:3] in used_prefixes:
            continue
        used_prefixes.add(w_lower[:3])
        result.append((w, score))
        if len(result) == result_size:
            break
    return result


In [17]:
import numpy as np
def get_b(model, target_word, filepath, result_size):
    base_vec = model[target_word]
    base_norm = base_vec / np.linalg.norm(base_vec)
    with open(filepath, "r", encoding="utf-8") as file:
        candidates = file.read().split()
    top_results = []
    for w in candidates:
        if w == target_word or w not in model:
            continue
        vec = model[w]
        vec_norm = vec / np.linalg.norm(vec)
        similarity = float(np.dot(base_norm, vec_norm))
        if len(top_results) < result_size:
            top_results.append((w, similarity))
            top_results.sort(key=lambda x: x[1], reverse=True)
        else:
            if similarity > top_results[-1][1]:
                top_results[-1] = (w, similarity)
                top_results.sort(key=lambda x: x[1], reverse=True)
    return top_results


In [18]:
word = "маяк"

res_a = get_a(wv_embeddings, word, 60, 10)
res_b = get_b(wv_embeddings, word, "text.txt", 10)


print("Результат а")
for r in res_a:
    print(r)

print("\nРезультат б")
for r in res_b:
   print(r)

Результат а
('маячный', 0.6048486828804016)
('мыс', 0.5482443571090698)
('утес', 0.5242083072662354)
('скала', 0.5140703320503235)
('башня', 0.5124934315681458)
('обелиски', 0.5116521716117859)
('пристань', 0.5099610090255737)
('береговый', 0.5064749717712402)
('мостик', 0.5064305663108826)
('шпиль', 0.5039313435554504)

Результат б
('светомаяк', 0.5747745633125305)
('радиомаяк', 0.5415729284286499)
('утес', 0.5242083668708801)
('скала', 0.5140702724456787)
('башня', 0.5124935507774353)
('пристань', 0.5099611878395081)
('световоймаяк', 0.4968123435974121)
('корабль', 0.4957634210586548)
('берег', 0.48608696460723877)
('обсерватория', 0.4761872887611389)
